In [2]:
!pip install pyhive

    100% |################################| 51kB 4.3MB/s eta 0:00:01
    100% |################################| 829kB 731kB/s eta 0:00:01
  Running setup.py bdist_wheel for pyhive ... done
  Stored in directory: /root/.cache/pip/wheels/0b/8d/51/4f4d0799c9281b8f6d742795733395638b5c2f490cf49dc09a
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built pyhive future
You are using pip version 9.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from pyhive import presto
import pandas as pd

In [20]:
import json
import pandas as pd
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
import warnings

In [5]:
presto_conn = presto.connect(
     host='presto',
     port=8080,
     catalog='hive',
     schema='default'
 )

In [15]:
query = "SHOW TABLES"
tables = pd.read_sql_query(query, presto_conn)
tables

,Table
0,the_games


In [12]:
query = "SELECT hometeam FROM the_games"
tables = pd.read_sql_query(query, presto_conn)
tables

DatabaseError: {'errorCode': 65536, 'errorName': 'INTERNAL_ERROR', 'errorType': 'INTERNAL_ERROR', 'failureInfo': {'type': 'java.lang.NullPointerException', 'suppressed': [], 'stack': ['com.facebook.presto.spi.RecordPageSource.getNextPage(RecordPageSource.java:124)', 'com.facebook.presto.operator.TableScanOperator.getOutput(TableScanOperator.java:246)', 'com.facebook.presto.operator.Driver.processInternal(Driver.java:380)', 'com.facebook.presto.operator.Driver.processFor(Driver.java:303)', 'com.facebook.presto.execution.SqlTaskExecution$DriverSplitRunner.processFor(SqlTaskExecution.java:577)', 'com.facebook.presto.execution.TaskExecutor$PrioritizedSplitRunner.process(TaskExecutor.java:529)', 'com.facebook.presto.execution.TaskExecutor$Runner.run(TaskExecutor.java:665)', 'java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1142)', 'java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:617)', 'java.lang.Thread.run(Thread.java:745)']}}

In [16]:
presto_cur = presto_conn.cursor()
presto_cur.execute('SELECT * FROM default.the_games')

In [17]:
df = pd.DataFrame(presto_cur.fetchall())

ConnectionError: HTTPConnectionPool(host='presto', port=8080): Max retries exceeded with url: /v1/statement/20211206_034525_00009_ar776/1 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x7f6df4c2da58>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [72]:
df

""


In [19]:
df = spark.read.parquet('/tmp/games')

df.registerTempTable('games')

In [48]:
query = """
create external table games
  stored as parquet
  location '/tmp/games'
  as
  select * from games
"""

spark.sql(query)

Py4JJavaError: An error occurred while calling o23.sql.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply$mcV$sp(FileFormatWriter.scala:215)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:173)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:173)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:65)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:173)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:145)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	at org.apache.

In [22]:
df.show()

+--------------------+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|            raw_game|  GameKey|Week|AwayTeam|AwayScore|HomeTeam|HomeScore|PointSpread|OverUnder|AwayTeamMoneyLine|HomeTeamMoneyLine|
+--------------------+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|{"TimeRemaining":...|202110802|   8|     CAR|       30|     ATL|       20|       -2.4|     74.1|              169|             -202|
|{"TimeRemaining":...|202110804|   8|     MIA|       17|     BUF|       41|      -22.9|     76.4|             1032|            -1592|
|{"TimeRemaining":...|202110806|   8|      SF|       52|     CHI|       35|        9.9|     63.0|             -325|              271|
|{"TimeRemaining":...|202110808|   8|     PIT|       24|     CLE|       16|       -8.7|     67.8|              282|             -344|
|{"TimeRemaining":...|202110811|   8|     PHI|       69|     D

In [24]:
df.count()

52

In [57]:
CAR = df[df.AwayTeam.isin("CAR")]
CAR

DataFrame[raw_game: string, GameKey: string, Week: string, AwayTeam: string, AwayScore: int, HomeTeam: string, HomeScore: int, PointSpread: float, OverUnder: float, AwayTeamMoneyLine: int, HomeTeamMoneyLine: int]

In [30]:
away = df.groupby("AwayTeam")

In [31]:
away

In [48]:
df(0)[4]

TypeError: 'DataFrame' object is not callable

In [58]:
a = 'https://api.sportsdata.io/v3/nfl/scores/json/ScoresByDate/2021-OCT-31'

In [60]:
a + '1'

'https://api.sportsdata.io/v3/nfl/scores/json/ScoresByDate/2021-OCT-311'

In [63]:
a = 'https://api.sportsdata.io/v3/nfl/scores/json/GameStatsByWeek/2021REG/'

In [64]:
a + '1'

'https://api.sportsdata.io/v3/nfl/scores/json/GameStatsByWeek/2021REG/1'

In [67]:
new = spark.read.parquet('/tmp/games1')

new.registerTempTable('games1')

In [81]:
HML = new.sort("HomeTeamMoneyLine", ascending = False)
HML.show()

+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|  GameKey|Week|AwayTeam|AwayScore|HomeTeam|HomeScore|PointSpread|OverUnder|AwayTeamMoneyLine|HomeTeamMoneyLine|
+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|  

In [78]:
spread = new.sort("PointSpread", ascending = False)
spread.show()

+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|  GameKey|Week|AwayTeam|AwayScore|HomeTeam|HomeScore|PointSpread|OverUnder|AwayTeamMoneyLine|HomeTeamMoneyLine|
+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|             1266|
|202110813|   8|     LAR|       60|     HOU|       35|       26.0|     74.1|            -2060|  

In [74]:
CAR_away = new[new.AwayTeam.isin("CAR")]
CAR_away.show()

+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|  GameKey|Week|AwayTeam|AwayScore|HomeTeam|HomeScore|PointSpread|OverUnder|AwayTeamMoneyLine|HomeTeamMoneyLine|
+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|202110802|   8|     CAR|       30|     ATL|       20|       -2.4|     74.1|              169|             -202|
|202110802|   8|     CAR|       30|     ATL|       20|       -2.4|     74.1|              169|             -202|
|202110802|   8|     CAR|       30|     ATL|       20|       -2.4|     74.1|              169|             -202|
|202110802|   8|     CAR|       30|     ATL|       20|       -2.4|     74.1|              169|             -202|
|202110802|   8|     CAR|       30|     ATL|       20|       -2.4|     74.1|              169|             -202|
|202110802|   8|     CAR|       30|     ATL|       20|       -2.4|     74.1|              169|  

In [72]:
CAR_home = new[new.HomeTeam.isin("CAR")]
CAR_home.show()

+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|  GameKey|Week|AwayTeam|AwayScore|HomeTeam|HomeScore|PointSpread|OverUnder|AwayTeamMoneyLine|HomeTeamMoneyLine|
+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|202110105|   1|     NYJ|        7|     CAR|       13|       -2.8|     31.1|             null|             null|
|202110105|   1|     NYJ|       18|     CAR|       24|       -5.1|     57.2|              198|             -239|
|202110205|   2|      NO|       11|     CAR|       33|        4.8|     56.6|             -194|              165|
|202110505|   5|     PHI|       27|     CAR|       23|       -3.2|     58.5|              151|             -179|
|202110605|   6|     MIN|       43|     CAR|       36|        3.2|     57.8|             -168|              142|
|202110905|   9|      NE|       31|     CAR|       10|        5.7|     53.4|             -229|  

In [80]:
K_home = new[new.HomeTeam.isin("KC")]
K_home.show()

+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|  GameKey|Week|AwayTeam|AwayScore|HomeTeam|HomeScore|PointSpread|OverUnder|AwayTeamMoneyLine|HomeTeamMoneyLine|
+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|202110116|   1|     CLE|       20|      KC|       23|       -3.8|     37.7|             null|             null|
|202110116|   1|     CLE|       37|      KC|       42|       -7.0|     69.3|              250|             -305|
|202110316|   3|     LAC|       38|      KC|       31|       -8.9|     68.6|              315|             -388|
|202110516|   5|     BUF|       48|      KC|       25|       -3.2|     71.8|              158|             -187|
|202110816|   8|     NYG|       22|      KC|       25|      -13.3|     67.4|              503|             -656|
|202110916|   9|      GB|       11|      KC|       17|       -8.9|     61.0|              334|  